In [1]:
import tensorflow as tf
import _pickle as cPickle
import numpy as np
import tensorflow.contrib.slim as slim


In [2]:
alldata = cPickle.load(open('notMNIST.pickle', 'rb'))
test_set = alldata['test_dataset'].reshape(-1,28,28,1)
test_set.shape

(10000, 28, 28, 1)

In [3]:
writer = tf.python_io.TFRecordWriter('train.tfrecords')
row_len = alldata['train_dataset'].shape[0]
train_label = alldata['train_labels']
img_size=28
depth=1
for i in range(row_len):
    img = alldata['train_dataset'][i].tostring()
    example = tf.train.Example(features=tf.train.Features(feature={
        'height': tf.train.Feature(int64_list=tf.train.Int64List(value=[img_size])),
        'width': tf.train.Feature(int64_list=tf.train.Int64List(value=[img_size])),
        'depth': tf.train.Feature(int64_list=tf.train.Int64List(value=[depth])),
        'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[int(train_label[i])])),
        'image_raw': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img]))}))
    writer.write(example.SerializeToString())
writer.close()

In [2]:
def convertToTFRecord(data, img_size, depth, label,name):
    writer = tf.python_io.TFRecordWriter(name+'.tfrecords')
    row_len = data.shape[0]
    for i in range(row_len):
        img = data[i].tostring()
        example = tf.train.Example(features=tf.train.Features(feature={
            'height': tf.train.Feature(int64_list=tf.train.Int64List(value=[img_size])),
            'width': tf.train.Feature(int64_list=tf.train.Int64List(value=[img_size])),
            'depth': tf.train.Feature(int64_list=tf.train.Int64List(value=[depth])),
            'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[int(label[i])])),
            'image_raw': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img]))}))
        writer.write(example.SerializeToString())
    writer.close()

In [11]:
convertToTFRecord(alldata['test_dataset'], 28, 1, alldata['test_labels'],'test')

In [3]:
def readTFRecords(filenames):
    tfrecord_file_queue = tf.train.string_input_producer(filenames, name='queue')
    reader = tf.TFRecordReader()
    _, tfrecord_serialized = reader.read(tfrecord_file_queue)
    tfrecord_features = tf.parse_single_example(tfrecord_serialized,
                        features={
                            'height': tf.FixedLenFeature([], tf.int64),
                            'width': tf.FixedLenFeature([], tf.int64),
                            'depth': tf.FixedLenFeature([], tf.int64),
                            'label': tf.FixedLenFeature([], tf.int64),
                            'image_raw': tf.FixedLenFeature([], tf.string),
                        }, name='features')
    image = tf.decode_raw(tfrecord_features['image_raw'], tf.float32)
    image = tf.reshape(image, [28,28,1])
    print (image.get_shape())
    label = tfrecord_features['label']
    label_oh = tf.one_hot(label,depth=10)
    return image, label_oh

In [5]:
def getcnn_model(input, isTraining=True):
    with slim.arg_scope([slim.conv2d, slim.fully_connected],
                      activation_fn=tf.nn.relu,
                      weights_initializer=tf.truncated_normal_initializer(0.0, 0.01),
                      weights_regularizer=slim.l2_regularizer(0.0005)):
        net = slim.repeat(images, 3, slim.conv2d, 64, [3, 3], scope='conv1')
        net = slim.flatten(net)
        net = slim.fully_connected(net, 10, activation_fn=None, scope='fc')
        return net

In [19]:
batch_size = 100
with tf.Graph().as_default():
    train_image, train_label = readTFRecords(['train.tfrecords'])
    images, labels = tf.train.shuffle_batch(
        [train_image, train_label],
        batch_size=batch_size,
        capacity=50000,
        min_after_dequeue=10000)
        #allow_smaller_final_batch=True)
    print (images.get_shape())
    net = getcnn_model(images)
    #predictions = tf.nn.softmax(net)
    loss = slim.losses.softmax_cross_entropy(net, labels)
    optimizer = tf.train.AdamOptimizer(1e-4)

    total_loss = slim.losses.get_total_loss()
    train_op = slim.learning.create_train_op(total_loss, optimizer)
    logdir = 'newd'
    slim.learning.train(train_op,logdir,number_of_steps=2000,save_summaries_secs=100,save_interval_secs=600)

(28, 28, 1)
(100, 28, 28, 1)
Instructions for updating:
Use tf.losses.softmax_cross_entropy instead. Note that the order of the logits and labels arguments has been changed.
Instructions for updating:
Use tf.losses.compute_weighted_loss instead.
Instructions for updating:
Use tf.losses.add_loss instead.
Instructions for updating:
Use tf.losses.get_total_loss instead.
Instructions for updating:
Use tf.losses.get_losses instead.
Instructions for updating:
Use tf.losses.get_regularization_losses instead.
INFO:tensorflow:Restoring parameters from newd\model.ckpt-1000
INFO:tensorflow:Starting Session.
INFO:tensorflow:Starting Queues.
INFO:tensorflow:global_step/sec: 0
INFO:tensorflow:global step 1001: loss = 0.4463 (9.79 sec/step)
INFO:tensorflow:global step 1002: loss = 0.8009 (0.45 sec/step)
INFO:tensorflow:global step 1003: loss = 0.8136 (0.50 sec/step)
INFO:tensorflow:global step 1004: loss = 0.7478 (0.50 sec/step)
INFO:tensorflow:global step 1005: loss = 0.6103 (0.48 sec/step)
INFO:ten

INFO:tensorflow:global step 1095: loss = 0.8320 (0.53 sec/step)
INFO:tensorflow:global step 1096: loss = 0.5371 (0.58 sec/step)
INFO:tensorflow:global step 1097: loss = 0.5364 (0.63 sec/step)
INFO:tensorflow:global step 1098: loss = 0.7638 (0.49 sec/step)
INFO:tensorflow:global step 1099: loss = 0.6473 (0.60 sec/step)
INFO:tensorflow:global step 1100: loss = 0.6537 (0.58 sec/step)
INFO:tensorflow:global step 1101: loss = 0.4132 (0.51 sec/step)
INFO:tensorflow:global step 1102: loss = 0.5019 (0.66 sec/step)
INFO:tensorflow:global step 1103: loss = 0.7791 (0.54 sec/step)
INFO:tensorflow:global step 1104: loss = 0.4977 (0.55 sec/step)
INFO:tensorflow:global step 1105: loss = 0.5797 (0.58 sec/step)
INFO:tensorflow:global step 1106: loss = 0.4293 (0.57 sec/step)
INFO:tensorflow:global step 1107: loss = 0.6251 (0.54 sec/step)
INFO:tensorflow:global step 1108: loss = 0.6794 (0.55 sec/step)
INFO:tensorflow:global step 1109: loss = 0.6422 (0.55 sec/step)
INFO:tensorflow:global step 1110: loss =

INFO:tensorflow:global step 1223: loss = 0.4385 (0.52 sec/step)
INFO:tensorflow:global step 1224: loss = 0.4894 (0.61 sec/step)
INFO:tensorflow:global step 1225: loss = 0.7433 (0.47 sec/step)
INFO:tensorflow:global step 1226: loss = 0.4964 (0.46 sec/step)
INFO:tensorflow:global step 1227: loss = 0.6571 (0.65 sec/step)
INFO:tensorflow:global step 1228: loss = 0.7041 (0.46 sec/step)
INFO:tensorflow:global step 1229: loss = 0.5608 (0.47 sec/step)
INFO:tensorflow:global step 1230: loss = 0.6023 (0.46 sec/step)
INFO:tensorflow:global step 1231: loss = 0.5991 (0.54 sec/step)
INFO:tensorflow:global step 1232: loss = 0.5833 (0.47 sec/step)
INFO:tensorflow:global step 1233: loss = 0.7705 (0.48 sec/step)
INFO:tensorflow:global step 1234: loss = 0.7345 (0.48 sec/step)
INFO:tensorflow:global step 1235: loss = 0.7648 (0.62 sec/step)
INFO:tensorflow:global step 1236: loss = 0.6158 (0.47 sec/step)
INFO:tensorflow:global step 1237: loss = 0.4852 (0.49 sec/step)
INFO:tensorflow:global step 1238: loss =

INFO:tensorflow:global step 1352: loss = 0.6456 (0.48 sec/step)
INFO:tensorflow:global step 1353: loss = 0.5649 (0.47 sec/step)
INFO:tensorflow:global step 1354: loss = 0.8154 (0.48 sec/step)
INFO:tensorflow:global step 1355: loss = 0.6113 (0.47 sec/step)
INFO:tensorflow:global step 1356: loss = 0.7893 (0.46 sec/step)
INFO:tensorflow:global step 1357: loss = 0.6807 (0.45 sec/step)
INFO:tensorflow:global step 1358: loss = 0.6161 (0.53 sec/step)
INFO:tensorflow:global step 1359: loss = 0.5414 (0.55 sec/step)
INFO:tensorflow:global step 1360: loss = 0.6649 (0.46 sec/step)
INFO:tensorflow:global step 1361: loss = 0.6082 (0.47 sec/step)
INFO:tensorflow:global step 1362: loss = 0.7437 (0.66 sec/step)
INFO:tensorflow:global step 1363: loss = 0.4819 (0.46 sec/step)
INFO:tensorflow:global step 1364: loss = 0.5778 (0.44 sec/step)
INFO:tensorflow:global_step/sec: 1.98
INFO:tensorflow:global step 1365: loss = 0.6424 (0.46 sec/step)
INFO:tensorflow:global step 1366: loss = 0.5760 (0.58 sec/step)
IN

INFO:tensorflow:global step 1480: loss = 0.4735 (0.46 sec/step)
INFO:tensorflow:global step 1481: loss = 0.5120 (0.45 sec/step)
INFO:tensorflow:global step 1482: loss = 0.5717 (0.63 sec/step)
INFO:tensorflow:global step 1483: loss = 0.5965 (0.48 sec/step)
INFO:tensorflow:global step 1484: loss = 0.5532 (0.46 sec/step)
INFO:tensorflow:global step 1485: loss = 0.6383 (0.47 sec/step)
INFO:tensorflow:global step 1486: loss = 0.4053 (0.66 sec/step)
INFO:tensorflow:global step 1487: loss = 0.7676 (0.48 sec/step)
INFO:tensorflow:global step 1488: loss = 0.6907 (0.47 sec/step)
INFO:tensorflow:global step 1489: loss = 0.6034 (0.49 sec/step)
INFO:tensorflow:global step 1490: loss = 0.7892 (0.46 sec/step)
INFO:tensorflow:global step 1491: loss = 0.6576 (0.51 sec/step)
INFO:tensorflow:global step 1492: loss = 0.6574 (0.48 sec/step)
INFO:tensorflow:global step 1493: loss = 0.7044 (0.47 sec/step)
INFO:tensorflow:global step 1494: loss = 0.5054 (0.46 sec/step)
INFO:tensorflow:global step 1495: loss =

INFO:tensorflow:global step 1608: loss = 0.7580 (0.55 sec/step)
INFO:tensorflow:global step 1609: loss = 0.4337 (0.50 sec/step)
INFO:tensorflow:global step 1610: loss = 0.4578 (0.54 sec/step)
INFO:tensorflow:global step 1611: loss = 0.5804 (0.48 sec/step)
INFO:tensorflow:global step 1612: loss = 0.6257 (0.52 sec/step)
INFO:tensorflow:global step 1613: loss = 0.6004 (0.54 sec/step)
INFO:tensorflow:global step 1614: loss = 0.5437 (0.49 sec/step)
INFO:tensorflow:global step 1615: loss = 0.7583 (0.51 sec/step)
INFO:tensorflow:global step 1616: loss = 0.4089 (0.54 sec/step)
INFO:tensorflow:global step 1617: loss = 0.4101 (0.56 sec/step)
INFO:tensorflow:global step 1618: loss = 0.6767 (0.57 sec/step)
INFO:tensorflow:global step 1619: loss = 0.6323 (0.60 sec/step)
INFO:tensorflow:global step 1620: loss = 0.4171 (0.55 sec/step)
INFO:tensorflow:global step 1621: loss = 0.6820 (0.54 sec/step)
INFO:tensorflow:global step 1622: loss = 0.6449 (0.47 sec/step)
INFO:tensorflow:global step 1623: loss =

INFO:tensorflow:global step 1737: loss = 0.4769 (0.49 sec/step)
INFO:tensorflow:global step 1738: loss = 0.6447 (0.56 sec/step)
INFO:tensorflow:global step 1739: loss = 0.6619 (0.48 sec/step)
INFO:tensorflow:global step 1740: loss = 0.7552 (0.46 sec/step)
INFO:tensorflow:global step 1741: loss = 0.4893 (0.59 sec/step)
INFO:tensorflow:global_step/sec: 1.81992
INFO:tensorflow:global step 1742: loss = 0.7773 (0.55 sec/step)
INFO:tensorflow:global step 1743: loss = 0.7085 (0.51 sec/step)
INFO:tensorflow:global step 1744: loss = 0.5069 (0.55 sec/step)
INFO:tensorflow:global step 1745: loss = 0.6000 (0.55 sec/step)
INFO:tensorflow:global step 1746: loss = 0.4729 (0.56 sec/step)
INFO:tensorflow:global step 1747: loss = 0.5649 (0.50 sec/step)
INFO:tensorflow:global step 1748: loss = 0.6015 (0.52 sec/step)
INFO:tensorflow:global step 1749: loss = 0.5950 (0.48 sec/step)
INFO:tensorflow:global step 1750: loss = 0.6435 (0.46 sec/step)
INFO:tensorflow:global step 1751: loss = 0.6620 (0.48 sec/step)

INFO:tensorflow:global step 1865: loss = 0.7712 (0.70 sec/step)
INFO:tensorflow:global step 1866: loss = 0.4274 (0.52 sec/step)
INFO:tensorflow:global step 1867: loss = 0.6902 (0.59 sec/step)
INFO:tensorflow:global step 1868: loss = 0.4586 (0.58 sec/step)
INFO:tensorflow:global step 1869: loss = 0.5361 (0.57 sec/step)
INFO:tensorflow:global step 1870: loss = 0.5216 (0.46 sec/step)
INFO:tensorflow:global step 1871: loss = 0.5649 (0.56 sec/step)
INFO:tensorflow:global step 1872: loss = 0.5358 (0.60 sec/step)
INFO:tensorflow:global step 1873: loss = 0.6625 (0.62 sec/step)
INFO:tensorflow:global step 1874: loss = 0.6730 (0.54 sec/step)
INFO:tensorflow:global step 1875: loss = 0.4697 (0.60 sec/step)
INFO:tensorflow:global step 1876: loss = 0.4683 (0.48 sec/step)
INFO:tensorflow:global step 1877: loss = 0.5730 (0.56 sec/step)
INFO:tensorflow:global step 1878: loss = 0.4824 (0.62 sec/step)
INFO:tensorflow:global step 1879: loss = 0.4446 (0.56 sec/step)
INFO:tensorflow:global step 1880: loss =

INFO:tensorflow:global step 1993: loss = 0.6375 (0.52 sec/step)
INFO:tensorflow:global step 1994: loss = 0.6839 (0.55 sec/step)
INFO:tensorflow:global step 1995: loss = 0.4411 (0.54 sec/step)
INFO:tensorflow:global step 1996: loss = 0.5016 (0.48 sec/step)
INFO:tensorflow:global step 1997: loss = 0.3977 (0.47 sec/step)
INFO:tensorflow:global step 1998: loss = 0.4126 (0.52 sec/step)
INFO:tensorflow:global step 1999: loss = 0.4281 (0.49 sec/step)
INFO:tensorflow:global step 2000: loss = 0.5452 (0.58 sec/step)
INFO:tensorflow:Stopping Training.
INFO:tensorflow:Finished training! Saving model to disk.


In [20]:
batch_size = 100
with tf.Graph().as_default():
    #train_image, train_label = readTFRecords(['test.tfrecords'])
    #images, labels = tf.train.batch(
    #    [train_image, train_label],
    #    batch_size=batch_size, allow_smaller_final_batch=True)
    #print (images.get_shape())
    images = x = tf.placeholder(tf.float32, shape=[None, 28,28,1])
    logits = getcnn_model(images, isTraining=False)
    preds = tf.argmax(logits, 1)
    #correct_preds = tf.equal(preds, tf.argmax(labels, 1))
    #accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))
    with tf.Session() as sess:
        variables_to_restore = slim.get_model_variables()
        init_assign_op, init_feed_dict = slim.assign_from_checkpoint('newd/model.ckpt-2000', variables_to_restore)
        print (init_assign_op)
        sess.run(init_assign_op,init_feed_dict)
        print ('Loading done')
        total = 0
        for ti in range(50):
            start = ti*200
            predis = sess.run([preds],feed_dict={images:test_set[start:start+200]})
            total += np.sum(np.equal(alldata['test_labels'][start:start+200], predis))
        print ('Accuracy is ',total/100)

name: "group_deps"
op: "NoOp"
input: "^Assign"
input: "^Assign_1"
input: "^Assign_2"
input: "^Assign_3"
input: "^Assign_4"
input: "^Assign_5"
input: "^Assign_6"
input: "^Assign_7"

Loading done
Accuracy is  92.32


180